Defining methods and importing libraries that will be used in Genetic Algorithm

In [ ]:
from numpy.random.mtrand import randint
import numpy as np
from numpy import *
from numpy import meshgrid
from random import choices
import random
import statistics
# from google.colab import drive
# drive.mount('/content/drive')

# F1: Shifted Sphere benchmark function implementation output. Individual[i] bounds are -100 to 100 according to cec2005.
def Sphere(Individual):
        d = len(Individual)
        res = 0.0
        for i in range(d):
            res += Individual[i] ** 2
        return res
# F2: Shifted Schwefel’s Problem. Individual[i] bounds are -100 to 100 according to cec2005.
def Schwefel(Individual):
  d = len(Individual)
  res = 0.0
  slicedSum = 0.0
  for i in Individual:
    slicedSum += i
    res += slicedSum ** 2
  return res

# F3: Shifted Rotated High Conditioned Elliptic benchmark function implementation output. Individual[i] bounds are -100 to 100 according to cec2005.
def Elliptic(Individual):
  c = 1.0e6
  d = len(Individual)
  res = 0.0
  for i in range(d):
    exp = float(i) / (d - 1)
    res += c ** exp * Individual[i] ** 2
  return res

# Here Each chromosome (Individual) is passed to benchmark function and that returns calculated (fitness) value.
# Actually fitness decreases every time when we approach global minimum. But we implemented it in reverse manner.
# Means if fitness is getting lesser show us some bigger value. Means As we approach global minimum fitness is increasing.
# To do that we implemented abs(1/result). This will make smaller values the bigger values.
def fitnessFunc(Individual, BenchMarkFunction):

  if BenchMarkFunction == 1:
    result  = Sphere(Individual)
  elif BenchMarkFunction == 2:
    result  = Schwefel(Individual)
  elif BenchMarkFunction == 3:
    result  = Elliptic(Individual)

  if result.any() == 0:
    return 100000
  else:
    return abs(1/result)

# Tournament selection. In this method it do tournament according to population and select best two parents according to fitness.
def Selection(popWithFitness):
  return choices(population= [item[1] for item in popWithFitness], weights= [item[0] for item in popWithFitness], k=2)

# This is one point crossover method. In this method 2 lists of genes (chromosomes) are cut into two parts then their second part is exchanged with each other.
def OnePointCrossOver(p1, p2):

  # This condition checks whether size of two chromosoms is same or not.
  if len(p1) != len(p2):
    raise ValueError("Genes of both parents should be equal.")

  #This condition checks if size of choromosome is less than two then dont do crossover but just renturn same parent.
  if len(p1) < 2:
    return p1, p2


  #chromosome cut point. That is chosen here as random point of array. Means it cuts the array at random point. Then swap their second part.
  GenomCutPoint = randint(1, len(p1) - 1)
  p1Crossed = np.concatenate((p1[0:GenomCutPoint], p2[GenomCutPoint:]))
  p2Crossed = np.concatenate((p2[0:GenomCutPoint], p1[GenomCutPoint:]))
  return p1Crossed, p2Crossed

# In the following method mutation is done. picking some random gene of a chromosome then multiply that with its one percent.
def Mutation(GeneofChild):
  ArrayIndex = random.randrange(len(GeneofChild))
  GeneofChild[ArrayIndex] = GeneofChild[ArrayIndex] * abs(GeneofChild[ArrayIndex]/100)
  return GeneofChild




Generating new random population and generating the generations by using Genetic technique.

In [ ]:
def Optimization(OptFunc):
  # Following is the total number of genes in one chromosome. i.e vector size
  ChormosomSize = 10

  # Following two parameters are Bounds of real numbers used in genes. Mean real numbers in genes are in following range.
  upperBound = 100
  lowerBound = -100

  # Following is population size. i.e total number of chromosome in one population.
  popSize = 10

  # In following function generating a gene. Means, a list of real numbers within bounds limit and size of list is equal to chromosome size. In this case it is 10.
  def genes():
    gene = np.random.uniform(lowerBound,upperBound, ChormosomSize)
    return gene

  # Making initial population here. Appending chromosome in a list. Means appending lists of real numbers in another list to form population. This makes 2D list.
  # In this we are generating population of size 10. chromosome of size 10 is added 10 times in a list. That makes it 10x10 list.
  def population():

    pop = np.empty(shape=(popSize,ChormosomSize))
    for i in range(popSize):
      pop[i] = genes()
    return pop

  initialPopulation = population()

  #This is a generations limit. Means, we will iterate 1000 times to find best solutions. we will finish after 1000 iterations either we find best or not.
  TotalGenerations = 1000
  for _ in range(TotalGenerations):
    # Here fitness values are assigned to each chromosome in the population with use of fitness function (i.e with the use of benchmark functions in fitness function).
    # You can change the value here to change the becnhmark function. i.e 1 for Sphere, 2 for Schwefel, 3 for Elliptic.
    popWithFitness = []
    for i in range(popSize):
      popWithFitness.append((fitnessFunc(initialPopulation[i], OptFunc), initialPopulation[i]))

    # Sorting the list with respect to fitness so that we get solutions with high fitness on top.
    popWithFitness.sort(reverse=True, key=lambda x: x[0])
    # print(popWithFitness)

    # Breaks the loop if we find some good solution
    if popWithFitness[0][0] > 99999:
      break

    #Best two solutions with respect to fitness for next generation. This is elitisam
    BestSolutions = [item[1] for item in popWithFitness[:2]]
    #print(BestSolutions)


    # This loop will iterate 4 times and generate 4 best-fitness pairs of parents for next generation.
    for i in range(int(len(initialPopulation)/2 -1)):
      #Tournament selection. Following choices function will do a random tournament and return the best two parents of that tournament.
      SelectedParentsInTournament = Selection(popWithFitness)

      #Crossover of two best parants gives birth to two child.
      C1, C2 = OnePointCrossOver(SelectedParentsInTournament[0], SelectedParentsInTournament[1])

      # Mutation function is called for mutation.
      C1Mutated = Mutation(C1)
      C2Mutated = Mutation(C2)

      # Adding new solutions into a BestSolutions list.
      BestSolutions.append(C1Mutated)
      BestSolutions.append(C2Mutated)
      #print(BestSolutions)

    # Assigning BestSolutions list to initial population. Means, our new population (generation) is now Best solutions of previous generation.
    initialPopulation = BestSolutions
  return initialPopulation



Getting mean rasult of 10 runs on all three benchmark functions.

In [ ]:
Best1SphereSolsFromTenResults = []
Best1SchewefelSolsFromTenResults = []
Best1EllipticSolsFromTenResults = []
for _ in range(10):
  SphereBenchMark = Optimization(1)
  #taking first (Best) solution amongst all.
  Best1SphereSolsFromTenResults.append(SphereBenchMark[0])

  SchwefelBenchMark = Optimization(2)
  Best1SchewefelSolsFromTenResults.append(SchwefelBenchMark[0])

  EllipticBenchMark = Optimization(3)
  Best1EllipticSolsFromTenResults.append(EllipticBenchMark[0])

MeanOfTenRunsOfShpere = []
MeanOfTenRunsOfSchwefel = []
MeanOfTenRunsOfElliptic = []
for i in range(10):

  xi = [item[i] for item in Best1SphereSolsFromTenResults]
  MeanOfTenRunsOfShpere.append(statistics.mean(xi))

  yi = [item[i] for item in Best1SchewefelSolsFromTenResults]
  MeanOfTenRunsOfSchwefel.append(statistics.mean(yi))

  zi = [item[i] for item in Best1EllipticSolsFromTenResults]
  MeanOfTenRunsOfElliptic.append(statistics.mean(zi))

print("Mean of top 1 solutions of 10 runs of Sphere Benchmark function \n")
print("Mean Result: ", MeanOfTenRunsOfShpere)
print("Solution of Mean result values: ",Sphere(MeanOfTenRunsOfShpere), "\n\n")

print("Mean of top 1 solutions of 10 runs of Schwefel Benchmark function \n")
print("Mean Result: ", MeanOfTenRunsOfSchwefel)
print("Solution of Mean result values: ",Schwefel(MeanOfTenRunsOfSchwefel), "\n\n")

print("Mean of top 1 solutions of 10 runs of Elliptic Benchmark function \n")
print("Mean Result: ", MeanOfTenRunsOfElliptic)
print("Solution of Mean result values: ",Elliptic(MeanOfTenRunsOfElliptic))

Mean of top 1 solutions of 10 runs of Sphere Benchmark function 

Mean Result:  [-0.0002519908753336376, -0.004252152728265897, 0.0008885078868018367, 0.00031114280843165364, -0.0006274759544526018, 0.0004032471594192916, -0.006198379567717247, 0.0005433138820389332, 0.0010876122805457589, 7.858987900511697e-06]
Solution of Mean result values:  5.9484954159289883e-05 


Mean of top 1 solutions of 10 runs of Schwefel Benchmark function 

Mean Result:  [-0.00938971942367282, 0.03289304291620037, -0.0019938738409893825, -0.010877878677726402, -0.0005719841176943097, -0.0008488473600110452, -2.849368486677967e-06, -0.00045708740439814454, -0.0004729239643083267, 0.001839508147753024]
Solution of Mean result values:  0.0017345393649833628 


Mean of top 1 solutions of 10 runs of Elliptic Benchmark function 

Mean Result:  [-0.008298656306136508, -0.0014333316340741345, 0.0002131840860683618, -9.023036236818668e-05, -0.00029561189230850463, 2.51448852266794e-05, -1.08197391244512e-07, 2.1558